In [185]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import shutil
import tensorflow as tf
import graphviz
import re

from datetime import datetime
from keras.layers import Dense
from keras.models import Sequential
from sklearn import linear_model
from sklearn import metrics
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import chi2
from sklearn.feature_selection import r_regression, SelectKBest
from sklearn.feature_selection import r_regression, SelectPercentile
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LogisticRegression, RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils._testing import ignore_warnings
from tensorflow import keras
from keras.callbacks import TensorBoard
from tabulate import tabulate
from xgboost import XGBClassifier

%matplotlib inline


In [186]:
pd.set_option('display.show_dimensions', True)
pd.set_option('display.max_columns', 200)


# Load Data

In [187]:
df = pd.read_csv('../Datasets/fifa2022 player stats.csv', parse_dates=['Joined'])
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,€107.5M,€250K,2341,Right,3.0,3.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos18"">CAM",18.0,2020-01-30,NaN,2025,179cm,69kg,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,NaN,73.0,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,€206.9M,72.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,€93M,€140K,2314,Right,4.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos11"">LDM",8.0,2018-07-01,NaN,2026,189cm,82kg,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,NaN,85.0,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,€160.4M,74.0
2,176580,L. Suárez,34,https://cdn.sofifa.com/players/176/580/22_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,88,88,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,€44.5M,€135K,2307,Right,5.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos24"">RS",9.0,2020-09-25,NaN,2022,182cm,83kg,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,NaN,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,€91.2M,42.0
3,192985,K. De Bruyne,30,https://cdn.sofifa.com/players/192/985/22_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/30.png,€125.5M,€350K,2304,Right,4.0,5.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos13"">RCM",17.0,2015-08-30,NaN,2025,181cm,70kg,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,NaN,65.0,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,€232.2M,68.0
4,224334,M. Acuña,29,https://cdn.sofifa.com/players/224/334/22_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,84,84,Sevilla FC,https://cdn.sofifa.com/teams/481/30.png,€37M,€45K,2292,Left,2.0,3.0,4.0,High/ High,Stocky (170-185),No,"<span class=""pos pos7"">LB",19.0,2020-09-14,NaN,2024,172cm,69kg,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,NaN,84.0,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,€77.7M,80.0


In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16710 entries, 0 to 16709
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID                        16710 non-null  int64         
 1   Name                      16710 non-null  object        
 2   Age                       16710 non-null  int64         
 3   Photo                     16710 non-null  object        
 4   Nationality               16710 non-null  object        
 5   Flag                      16710 non-null  object        
 6   Overall                   16710 non-null  int64         
 7   Potential                 16710 non-null  int64         
 8   Club                      16446 non-null  object        
 9   Club Logo                 16710 non-null  object        
 10  Value                     16710 non-null  object        
 11  Wage                      16710 non-null  object        
 12  Special           

In [189]:
df.describe(datetime_is_numeric=True)

,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Joined,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Overall Rating,DefensiveAwareness
count,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16684.000000,15198,16710.000000,16710.000000,16710.000000,16710.000000,16673.000000,16710.000000,16673.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16673.000000,16710.000000,16673.000000,16710.000000,16673.000000,16710.000000,16710.000000,16710.000000,16710.000000,16702.000000,16702.000000,16673.000000,16710.000000,16459.000000,892.000000,16710.000000,16673.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,16710.000000,15818.000000
mean,220560.467923,25.727409,67.646320,72.572292,1652.497307,1.169958,3.008199,2.475464,20.458643,2019-04-21 23:56:18.286616576,52.212448,48.725075,54.123339,61.314423,45.652972,58.591203,50.574522,45.074087,55.454877,61.387433,65.684919,65.783124,65.222875,63.592400,64.943262,60.642190,65.896659,64.204548,66.266008,49.989228,57.955536,48.057418,53.150461,56.378936,50.615919,61.001883,48.021300,49.155536,46.830564,15.683662,15.521364,15.457989,15.581927,15.789228,68.565410,48.019408
min,27.000000,16.000000,28.000000,38.000000,571.000000,1.000000,1.000000,1.000000,1.000000,1990-12-01 00:00:00,7.000000,3.000000,5.000000,8.000000,4.000000,5.000000,6.000000,4.000000,9.000000,8.000000,13.000000,15.000000,18.000000,28.000000,19.000000,12.000000,22.000000,13.000000,18.000000,4.000000,11.000000,4.000000,3.000000,10.000000,7.000000,12.000000,4.000000,3.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,3.000000
25%,203891.250000,22.000000,63.000000,69.000000,1525.000000,1.000000,3.000000,2.000000,8.000000,2018-07-01 00:00:00,42.000000,34.000000,46.000000,57.000000,33.000000,54.000000,39.000000,33.000000,47.000000,58.000000,58.000000,59.000000,57.000000,58.000000,57.000000,52.000000,59.000000,57.000000,59.000000,37.000000,47.000000,28.000000,44.000000,48.000000,41.000000,55.000000,27.000000,29.000000,26.000000,8.000000,8.000000,8.000000,8.000000,8.000000,65.000000,30.000000
50%,229253.000000,25.000000,68.000000,72.000000,1687.000000,1.000000,3.000000,2.000000,17.000000,2020-01-01 00:00:00,57.000000,53.000000,57.000000,64.000000,48.000000,63.500000,53.000000,44.000000,58.000000,65.000000,68.000000,68.000000,68.000000,64.000000,67.000000,62.000000,67.000000,67.000000,68.000000,55.000000,61.000000,55.000000,58.000000,58.000000,52.000000,62.000000,53.000000,57.000000,54.000000,11.000000,11.000000,11.000000,11.000000,11.000000,68.000000,54.000000
75%,245368.750000,29.000000,72.000000,76.000000,1826.000000,1.000000,3.000000,3.000000,27.000000,2021-01-21 00:00:00,65.000000,64.000000,66.000000,70.000000,59.000000,70.000000,64.000000,59.000000,66.000000,71.000000,76.000000,76.000000,75.000000,69.000000,75.000000,70.000000,74.000000,75.000000,75.000000,65.000000,71.000000,65.000000,67.000000,66.000000,62.000000,69.000000,67.000000,67.000000,65.000000,14.000000,14.000000,14.000000,14.000000,14.000000,73.000000,65.000000
max,264704.000000,54.000000,93.000000,95.000000,2341.000000,5.000000,5.000000,5.000000,99.000000,2021-09-28 00:00:00,94.000000,95.000000,93.000000,94.000000,90.000000,96.000000,94.000000,94.000000,93.000000,96.000000,97.000000,97.000000,96.000000,96.000000,96.000000,95.000000,95.000000,97.000000,97.000000,94.000000,95.000000,95.000000,96.000000,95.000000,96.000000,96.000000,94.000000,93.000000,95.000000,91.000000,92.000000,93.000000,93.000000,90.000000,93.000000,93.000000
std,38496.607959,5.048910,6.457695,5.688085,257.271696,0.485305,0.6

In [190]:
df.describe(include=np.object0)

,Name,Photo,Nationality,Flag,Club,Club Logo,Value,Wage,Preferred Foot,Work Rate,Body Type,Real Face,Position,Loaned From,Contract Valid Until,Height,Weight,Best Position,Release Clause
count,16710,16710,16710,16710,16446,16710,16710,16710,16710,16710,16681,16681,16684,1132,16359,16710,16710,16710,14961
unique,16089,16710,168,168,869,918,254,134,2,10,10,2,29,342,56,50,58,15,1170
top,J. Rodríguez,https://cdn.sofifa.com/players/212/198/22_60.png,England,https://cdn.sofifa.com/flags/gb-eng.png,AC Milan,https://cdn.sofifa.com/teams/47/30.png,€1.2M,€2K,Right,Medium/ Medium,Normal (170-185),No,"<span class=""pos pos28"">SUB","<a href=""/team/39/atalanta/"">Atalanta</a>",2022,180cm,70kg,CB,€1.1M
freq,10,1,1845,1845,47,47,586,2316,12613,7866,5713,13905,7192,17,4781,1286,1303,3214,452


# Fill NaNs

In [191]:
df['Club'].fillna('Unavailable', inplace=True)
df['Body Type'].fillna('Unavailable', inplace=True)
df['Real Face'].fillna('Unavailable', inplace=True)
df['Position'].fillna('Unavailable', inplace=True)
df['Jersey Number'].fillna(0, inplace=True)
df['Joined'].fillna(datetime.strptime('20220101', '%Y%m%d'), inplace=True)
df['Loaned From'].fillna('Not Loaned', inplace=True)
df['Contract Valid Until'].fillna(2030, inplace=True)
df['Volleys'].fillna(0.0, inplace=True)
df['Curve'].fillna(0.0, inplace=True)
df['Agility'].fillna(0.0, inplace=True)
df['Balance'].fillna(0.0, inplace=True)
df['Jumping'].fillna(0.0, inplace=True)
df['Interceptions'].fillna(0.0, inplace=True)
df['Positioning'].fillna(0.0, inplace=True)
df['Vision'].fillna(0.0, inplace=True)
df['Composure'].fillna(0.0, inplace=True)
df['Marking'].fillna(0.0, inplace=True)
df['SlidingTackle'].fillna(0.0, inplace=True)
df['Release Clause'].fillna(0.0, inplace=True)
df['DefensiveAwareness'].fillna(0.0, inplace=True)


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16710 entries, 0 to 16709
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID                        16710 non-null  int64         
 1   Name                      16710 non-null  object        
 2   Age                       16710 non-null  int64         
 3   Photo                     16710 non-null  object        
 4   Nationality               16710 non-null  object        
 5   Flag                      16710 non-null  object        
 6   Overall                   16710 non-null  int64         
 7   Potential                 16710 non-null  int64         
 8   Club                      16710 non-null  object        
 9   Club Logo                 16710 non-null  object        
 10  Value                     16710 non-null  object        
 11  Wage                      16710 non-null  object        
 12  Special           

# Drop Useless Columns

In [193]:
df.drop(['ID', 'Photo', 'Club Logo', 'Flag'], axis=1, inplace=True)

# Adjust Some Formats

In [194]:
def num_format(num_string):
    '''This function converts the Value, Wage, Height, and Weight columns to float numbers'''

    if isinstance(num_string, float):
        return num_string
    else:
        if num_string.endswith('M'):
            unit = 1000000
        elif num_string.endswith('K'):
            unit = 1000
        else:
            unit = 1
        
        num = re.findall(r'\d+\.?\d*', num_string)


        return float(float(num[0]) * int(unit))

In [195]:
def pos_format(pos):
    '''This function removes the <span class> and returns the player's position'''

    return(re.sub(r'<(.*?)>', '', pos))

In [196]:
def contract_valid_until(x):
    '''This function returns the year part only of the "Contract Valid Until" column'''

    if ',' in str(x):
        return x.split(',')[1]
    else:
        return x

In [197]:
df['Value'] = df['Value'].apply(num_format)
df['Wage'] = df['Wage'].apply(num_format)
df['Height'] = df['Height'].apply(num_format)
df['Weight'] = df['Weight'].apply(num_format)
df['Release Clause'] = df['Release Clause'].apply(num_format)

In [198]:
df['Position'] = df['Position'].apply(pos_format)
df['Loaned From'] = df['Loaned From'].apply(pos_format)

In [199]:
df['Jersey Number'] = df['Jersey Number'].astype('int')
df['Contract Valid Until'] = df['Contract Valid Until'].apply(contract_valid_until)
df['Contract Valid Until'] = df['Contract Valid Until'].astype('int')

# Final Review

In [200]:
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,Bruno Fernandes,26,Portugal,88,89,Manchester United,107500000.0,250000.0,2341,Right,3.0,3.0,4.0,High/ High,Unique,Yes,CAM,18,2020-01-30,Not Loaned,2025,179.0,69.0,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,0.0,73.0,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,206900000.0,72.0
1,L. Goretzka,26,Germany,87,88,FC Bayern München,93000000.0,140000.0,2314,Right,4.0,4.0,3.0,High/ Medium,Unique,Yes,LDM,8,2018-07-01,Not Loaned,2026,189.0,82.0,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,0.0,85.0,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,160400000.0,74.0
2,L. Suárez,34,Uruguay,88,88,Atlético de Madrid,44500000.0,135000.0,2307,Right,5.0,4.0,3.0,High/ Medium,Unique,Yes,RS,9,2020-09-25,Not Loaned,2022,182.0,83.0,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,0.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,91200000.0,42.0
3,K. De Bruyne,30,Belgium,91,91,Manchester City,125500000.0,350000.0,2304,Right,4.0,5.0,4.0,High/ High,Unique,Yes,RCM,17,2015-08-30,Not Loaned,2025,181.0,70.0,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,0.0,65.0,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,232200000.0,68.0
4,M. Acuña,29,Argentina,84,84,Sevilla FC,37000000.0,45000.0,2292,Left,2.0,3.0,4.0,High/ High,Stocky (170-185),No,LB,19,2020-09-14,Not Loaned,2024,172.0,69.0,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,0.0,84.0,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,77700000.0,80.0


In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16710 entries, 0 to 16709
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Name                      16710 non-null  object        
 1   Age                       16710 non-null  int64         
 2   Nationality               16710 non-null  object        
 3   Overall                   16710 non-null  int64         
 4   Potential                 16710 non-null  int64         
 5   Club                      16710 non-null  object        
 6   Value                     16710 non-null  float64       
 7   Wage                      16710 non-null  float64       
 8   Special                   16710 non-null  int64         
 9   Preferred Foot            16710 non-null  object        
 10  International Reputation  16710 non-null  float64       
 11  Weak Foot                 16710 non-null  float64       
 12  Skill Moves       

# Deep Learning

In [208]:
# encode the categorical data
cat_cols = [col for col in df.columns if df[col].dtype == 'object']
print(cat_cols)
oe = OrdinalEncoder()
df[cat_cols] = oe.fit_transform(df[cat_cols])

['Name', 'Nationality', 'Club', 'Preferred Foot', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Loaned From', 'Best Position']


In [209]:
%load_ext tensorboard

# create folders
now = datetime.now().strftime('%Y%m%d%H%M')
log_folder = f'Tensorboard Logs/Regression - Fifa 2022 Player Stats {now}'

# callbacks for TensorBoard
callbacks = TensorBoard(log_dir=log_folder, histogram_freq=1)

In [216]:
# we need to convert all datetime columns to float
dt_cols = [col for col in df.columns if df[col].dtype == 'datetime64[ns]']
df[dt_cols] = df[dt_cols].astype(np.int64)


In [217]:
# x and y
x = df.drop('Best Overall Rating', axis=1).values
y = df['Best Overall Rating'].values

# need to scale down x values because otherwise the model returns nan mse
sc = StandardScaler()
x = sc.fit_transform(x)

In [220]:
x.shape

(16710, 60)

In [218]:
# clear previous sessions
keras.backend.clear_session()

# create keras model
model = keras.models.Sequential([
    keras.layers.Dense(15, activation='relu', input_dim=x.shape[1]),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                915       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,094
Trainable params: 1,094
Non-trainable params: 0
_________________________________________________________________


2022-11-25 11:21:22.320258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [219]:
# use MAE instead of MSE for smaller values
model.compile(optimizer='adam', loss='MeanAbsoluteError',
              metrics=['MeanAbsoluteError'])

model.fit(x=x, y=y, validation_split=0.2,
          epochs=100, callbacks=callbacks)

Epoch 1/100
418/418 [==============================] - 2s 2ms/step - loss: 27.3020 - mean_absolute_error: 27.3020 - val_loss: 30.0746 - val_mean_absolute_error: 30.0746
Epoch 2/100
418/418 [==============================] - 1s 1ms/step - loss: 5.1605 - mean_absolute_error: 5.1605 - val_loss: 30.2641 - val_mean_absolute_error: 30.2641
Epoch 3/100
418/418 [==============================] - 1s 1ms/step - loss: 3.9668 - mean_absolute_error: 3.9668 - val_loss: 28.8983 - val_mean_absolute_error: 28.8983
Epoch 4/100
418/418 [==============================] - 1s 1ms/step - loss: 3.1859 - mean_absolute_error: 3.1859 - val_loss: 26.0148 - val_mean_absolute_error: 26.0148
Epoch 5/100
418/418 [==============================] - 1s 1ms/step - loss: 2.6462 - mean_absolute_error: 2.6462 - val_loss: 22.1802 - val_mean_absolute_error: 22.1802
Epoch 6/100
418/418 [==============================] - 1s 1ms/step - loss: 2.2514 - mean_absolute_error: 2.2514 - val_loss: 18.4129 - val_mean_absolute_error: 18.4